# Créations du dataset des Fingerprints à partir de la database originale

In [1]:
#Ensemble des importations requises
from aizynthfinder.chem import Molecule
import pandas as pd
import tqdm.notebook as tnote
import numpy as np
import os
import math

In [2]:
filename = "inserer_nom_fichier_a_transformer_en_fp"

In [11]:
#Lecture du fichier
data = pd.read_csv(filename) #, delimiter=";")

#Eventuellement, ajouter la partie commentée

In [6]:
def create_fingerprint(reactants, products):
    '''
    Renvoie le fingerprint du produit et celui de la reaction à partir des réactifs et produits au format smiles
    '''
    prd_f = Molecule(smiles=products).fingerprint(2)
    rea_f = sum(Molecule(smiles=x).fingerprint(2) for x in reactants.split("."))
    return prd_f, prd_f  - rea_f

In [7]:
def string_nonzero_from_fp(fp):
    '''
    Encode le fingerprint sous la forme d'une chaine de caracteres contenant les indices non nuls du fingerprint
    ainsi que les valeurs associées quand elles sont différentes de 1
    '''
    s = ""
    l = np.nonzero(fp)[0]
    for k in l:
        if (int(fp[k])!=1):
            s+= str(int(k))+"-"+str(int(fp[k]))
        else:
            s += str(int(k))
        s+= " "
    return s

In [9]:
def create_data_fingerprint(dataset):
    '''
    Création du dataset des fingerprints à partir du dataset original
    '''
    res_data = pd.DataFrame(columns=["Product_Fingerprint", "Reaction_Fingerprint"])
    for i in range(dataset.shape[0]):
        smarts = dataset.iloc[i,1], dataset.iloc[i,2]
        #On récupère les fingerprints et on les transforme en string selon notre format
        prd_f, rxn_f = create_fingerprint(smarts[0],smarts[1])
        prd_f, rxn_f = string_nonzero_from_fp(prd_f), string_nonzero_from_fp(rxn_f)
        res_data.loc[i] = [prd_f, rxn_f]
    return res_data

In [13]:
#On séquence la création de la database pour des questions de temps d'execution 
l = []
p = data.shape[0]//200
print(p)
pb = tnote.tqdm(total = p+1)
for i in range(p+1):
    l.append(create_data_fingerprint(data[i*200:(i+1)*200]))
    l[-1].to_csv(f"fichier_desire_", index = True)
    pb.update(1)

1309


  0%|          | 0/1310 [00:00<?, ?it/s]

In [14]:
res_data_ = pd.concat(l, ignore_index=True)

In [15]:
res_data_.to_csv("False_Reactions_Recommender_Fingerprints.csv")